In [34]:
from os.path import join
import pandas as pd
from pandas import Series, DataFrame
import roamability as rb

downloads = r'c:\Users\balob\Downloads\REPORT_OCS_DMI_SUBSCRIBERS_COMPARE'
dmi_subs_file_name = 'export_dmi_subs_190221.csv'
ocs_subs_file_name = 'export_ocs_subs_190227.csv'

In [11]:
# Получить абонентов DMI

df_dmi = pd.read_csv(join(downloads, dmi_subs_file_name), dtype='str')
df_dmi.head(3)

,MVNO_NAME,IMSI,MSISDN,S_IMSI,PROF_ID,SP_NAME
0,Cellact,425180020000000,972557012345,425019620010108,1,Partner_Combined
1,Cellact,425180020000000,972557012345,260060140068752,2,P4_Combined
2,Cellact,425180020000002,NaN,425019620011335,1,Partner_Combined


In [5]:
df_dmi.dtypes

MVNO_NAME    object
IMSI         object
MSISDN       object
S_IMSI       object
PROF_ID      object
SP_NAME      object
dtype: object

In [38]:
# Получить абонентов OCS (из базы данных)

#str_v1
sql_srt='''
SELECT si.IMSI
,mi.IMSI AS S_IMSI
,ss.STATUS, ss.START_DATE
FROM SUBSCRIBER_IMSIS si
LEFT JOIN MULTI_IMSI mi ON si.subscriber_id = mi.subscriber_id
LEFT JOIN SUBSCRIBER_STATUS ss ON si.subscriber_id = ss.subscriber_id
WHERE ss.STATUS IN ('Active', 'Suspended', 'Pending') AND ss.END_DATE IS NULL
'''

#str_v2 with RESELLER_NAME and ACCOUNT_NAME
sql_srt='''
SELECT r.RESELLER_NAME
,a.ACCOUNT_NAME
,si.IMSI
,mi.IMSI AS S_IMSI
,ss.STATUS, ss.START_DATE
FROM SUBSCRIBER_IMSIS si
LEFT JOIN MULTI_IMSI mi ON si.subscriber_id = mi.subscriber_id
LEFT JOIN SUBSCRIBER_STATUS ss ON si.subscriber_id = ss.subscriber_id
LEFT JOIN SUBSCRIBERS s ON si.subscriber_id = s.subscriber_id
LEFT JOIN ACCOUNTS a ON a.account_id = s.account_id
LEFT JOIN RESELLERS r ON r.reseller_id = a.reseller_id
WHERE ss.STATUS IN ('Active', 'Suspended', 'Pending') AND ss.END_DATE IS NULL
'''

with rb.MssqlConnect('172.18.11.82', '10028', 'BSS', 'iKQVm40AZAmyRaw72LeY') as cnxn: # prod connection
#with rb.MssqlConnect('172.20.39.14', '10028', 'ocslab', 'P9fznRUzyDSNO43j') as cnxn: # lab connection
    df_ocs = pd.read_sql_query(sql_srt, cnxn, coerce_float=False)
df_ocs.head(3)

,RESELLER_NAME,ACCOUNT_NAME,IMSI,S_IMSI,STATUS,START_DATE
0,TinkLabs RT,Tinklabs Simcards,425019613009763,425019613009763,Suspended,2017-12-21 11:18:36.393
1,TinkLabs RT,Tinklabs Simcards,425019613009764,425019613009764,Suspended,2017-12-21 11:25:44.607
2,TinkLabs RT,Tinklabs Simcards,425019613009765,425019613009765,Suspended,2017-12-21 11:38:56.397


In [39]:
# df_ocs.to_csv(join(downloads, 'export_ocs_subs_190227.csv'), index=False)

In [35]:
# Получить абонентов OCS (из файла)

df_ocs = pd.read_csv(join(downloads, ocs_subs_file_name), dtype='str')
df_ocs.head(3)

,Unnamed: 0,RESELLER_NAME,ACCOUNT_NAME,IMSI,S_IMSI,STATUS,START_DATE
0,0,Porto Seguro DMI RT,Porto Seguro Conecta,724540540263237,425019620000023,Active,2016-09-21 00:19:07.907
1,1,Porto Seguro DMI RT,Porto Seguro Conecta,724540540263250,425019620000025,Active,2016-09-21 16:50:24.880
2,2,Porto Seguro DMI RT,Porto Seguro Conecta,724540540263257,425019620000031,Active,2016-09-29 21:49:12.187


In [9]:
df_ocs.dtypes

RESELLER_NAME            object
ACCOUNT_NAME             object
IMSI                     object
S_IMSI                   object
STATUS                   object
START_DATE       datetime64[ns]
dtype: object

In [40]:
# Объединить данные OCS и DMI

df_ocs_dmi = pd.merge(df_dmi[df_dmi.MSISDN.notnull()], df_ocs, how='outer', on=['IMSI', 'S_IMSI'])
df_ocs_dmi.head(3)

,MVNO_NAME,IMSI,MSISDN,S_IMSI,PROF_ID,SP_NAME,RESELLER_NAME,ACCOUNT_NAME,STATUS,START_DATE
0,Cellact,425180020000000,972557012345,425019620010108,1,Partner_Combined,Cellact,Cellact Test sim cards,Active,2018-12-03 09:34:31.007
1,Cellact,425180020000000,972557012345,260060140068752,2,P4_Combined,Cellact,Cellact Test sim cards,Active,2018-12-03 09:34:31.007
2,Cellact,425180020000005,9725549151567,425019620012610,1,Partner_Combined,Cellact,Cellact Test sim cards,Active,2019-02-22 13:56:47.700


In [37]:
# Case 1. DMI-y; OCS-n

df_ocs_n = df_ocs_dmi[(df_ocs_dmi.STATUS.isnull())]
df_ocs_n.head(3)

,MVNO_NAME,IMSI,MSISDN,S_IMSI,PROF_ID,SP_NAME,Unnamed: 0,RESELLER_NAME,ACCOUNT_NAME,STATUS,START_DATE
54,Clay,425019613016521,447781518814,425019613016521,1,Partner_Clay_Com,NaN,NaN,NaN,NaN,NaN
55,Clay,425019613016521,447781518814,260060149016521,2,P4_Clay_Combined,NaN,NaN,NaN,NaN,NaN
112,Clay,425019613016550,447781518843,425019613016550,1,Partner_Clay_Com,NaN,NaN,NaN,NaN,NaN


In [9]:
df_ocs_n.MVNO_NAME.value_counts()

Tinklabs        8045
Telzar           155
PuertoSeguro      73
Nextel            48
Roamability       14
STI               10
Rewicom            7
Mondicon           6
Clay               4
Cellact            2
RUSEC              2
MultiByte          2
Name: MVNO_NAME, dtype: int64

In [88]:
df_ocs_n[(df_ocs_n.MVNO_NAME == 'Telzar') & (df_ocs_n.SP_NAME == 'P4_telzar')]

,MVNO_NAME,IMSI,MSISDN,S_IMSI,PROF_ID,SP_NAME,STATUS
120095,Telzar,425191200959682,972559997281,260060149000082,2,P4_telzar,NaN
137750,Telzar,425191201080917,972559389785,260060140002440,2,P4_telzar,NaN
141750,Telzar,425191201084621,972532446062,260060140081190,2,P4_telzar,NaN
180025,Telzar,425191201120512,972550058811,260060140002434,2,P4_telzar,NaN
180183,Telzar,425191201120661,972550058960,260060140002433,2,P4_telzar,NaN
246927,Telzar,425191201198928,972559347208,260060140002436,2,P4_telzar,NaN
248798,Telzar,425191201200761,972559349041,260060140000987,2,P4_telzar,NaN
422055,Telzar,425191201493503,972559226499,260060140000627,2,P4_telzar,NaN
460221,Telzar,425191201534974,972559775049,260060140001296,2,P4_telzar,NaN
461736,Telzar,425191201536182,972559770216,260060140002791,2,P4_telzar,NaN


In [117]:
# Методика решения проблемы Case 1. DMI-y; OCS-n

sql_srt='''
SELECT
si.subscriber_id, si.IMSI, COUNT(*) AS NUM
FROM SUBSCRIBER_IMSIS si
LEFT JOIN MULTI_IMSI mi ON si.subscriber_id=mi.subscriber_id
LEFT JOIN SUBSCRIBER_STATUS ss ON si.subscriber_id = ss.subscriber_id
WHERE ss.STATUS = 'Active' AND ss.END_DATE IS NULL
AND si.imsi IN ('''\
+ str(df_ocs_n[(df_ocs_n.MVNO_NAME == 'Telzar') & (df_ocs_n.SP_NAME == 'P4_telzar')].IMSI.tolist())[1:-1]\
+ ') GROUP BY si.subscriber_id, si.imsi HAVING COUNT(*) = 1'

with rb.MssqlConnect('172.18.11.82', '10028', 'BSS', 'iKQVm40AZAmyRaw72LeY') as cnxn: # prod connection
    df = pd.read_sql_query(sql_srt, cnxn, coerce_float=False)

pd.merge(df_ocs_n[(df_ocs_n.MVNO_NAME == 'Telzar') & (df_ocs_n.SP_NAME == 'P4_telzar')], df, how='inner', on='IMSI')

,MVNO_NAME,IMSI,MSISDN,S_IMSI,PROF_ID,SP_NAME,STATUS,subscriber_id,NUM
0,Telzar,425191201080917,972559389785,260060140002440,2,P4_telzar,NaN,260549,1
1,Telzar,425191201084621,972532446062,260060140081190,2,P4_telzar,NaN,264253,1
2,Telzar,425191201120512,972550058811,260060140002434,2,P4_telzar,NaN,300144,1
3,Telzar,425191201120661,972550058960,260060140002433,2,P4_telzar,NaN,300293,1
4,Telzar,425191201198928,972559347208,260060140002436,2,P4_telzar,NaN,364789,1
5,Telzar,425191201200761,972559349041,260060140000987,2,P4_telzar,NaN,366622,1
6,Telzar,425191201493503,972559226499,260060140000627,2,P4_telzar,NaN,545990,1
7,Telzar,425191201534974,972559775049,260060140001296,2,P4_telzar,NaN,593270,1
8,Telzar,425191201536182,972559770216,260060140002791,2,P4_telzar,NaN,594478,1
9,Telzar,425191201538589,972559761855,260060140002435,2,P4_telzar,NaN,596885,1


In [120]:
pd.merge(df_ocs_n[(df_ocs_n.MVNO_NAME == 'Telzar') & (df_ocs_n.SP_NAME == 'P4_telzar')], df, how='inner', on='IMSI')\
[['subscriber_id','IMSI','S_IMSI','MVNO_NAME']].to_csv(join(downloads,'Case1.csv'), index=False)

In [ ]:
# Далее по методике c:\W_DATA_ROAM\ПРОЕКТЫ\REPORT_OCS_DMI_SUBSCRIBERS_COMPARE\FIX_MISSED_P4_IMSI\

In [129]:
# Case 1. DMI-y; OCS-n. У абонента нет в OCS ни доного IMSI (частный случай)

df_ocs_dmi[(df_ocs_dmi.IMSI.notnull()) & (df_ocs_dmi.S_IMSI.isnull())].head()

,MVNO_NAME,IMSI,MSISDN,S_IMSI,PROF_ID,SP_NAME,STATUS
1362995,NaN,425191201134053,NaN,NaN,NaN,NaN,Active
1364634,NaN,724540540268812,NaN,NaN,NaN,NaN,Active
1364843,NaN,240400000030323,NaN,NaN,NaN,NaN,Active
1365035,NaN,724540540339447,NaN,NaN,NaN,NaN,Active


In [44]:
# Case 2. DMI-n; OCS-y

df_dmi_n = df_ocs_dmi[(df_ocs_dmi.IMSI.notnull()) & (df_ocs_dmi.MVNO_NAME.isnull())]
df_dmi_n.head(3)

,MVNO_NAME,IMSI,MSISDN,S_IMSI,PROF_ID,SP_NAME,RESELLER_NAME,ACCOUNT_NAME,STATUS,START_DATE
1361792,NaN,425019613010024,NaN,234553100900104,NaN,NaN,TinkLabs RT,Tinklabs Simcards,Active,2018-02-03 00:07:22.510
1361793,NaN,99999425019620000100,NaN,425019620000100,NaN,NaN,Cloud 9 TAP,Cloud 9 DMI Subscribers,Active,2016-11-10 20:31:25.167
1361794,NaN,99999425019620000101,NaN,425019620000101,NaN,NaN,Cloud 9 TAP,Cloud 9 DMI Subscribers,Active,2016-11-10 20:31:46.070


In [48]:
df_dmi_n.groupby([df_dmi_n.RESELLER_NAME, df_dmi_n.ACCOUNT_NAME], as_index=False)['IMSI'].count()

,RESELLER_NAME,ACCOUNT_NAME,IMSI
0,Cellact,Cellact Test sim cards,2
1,Cloud 9 TAP,Cloud 9 DMI Subscribers,11
2,KnowRoaming IMSI-RANGE TAP,test_kr,1
3,Maxcom RT,Celmax-Soriana,2
4,Maxcom RT,Maxcom Test Sim,4
5,Mobileye,Mobileye test simcards,10
6,Mondicon Relay,Mondicon IMSI Range Real Time (Relay),134
7,Mondicon Relay,Mondicon IMSI Range SIM Cards,739
8,Mondicon SIM RT,Mondicon RB Simcards,1
9,MultiByte RB Simcards,MultiByte Test Sim cards,1


In [46]:
df_dmi_n[(df_dmi_n.ACCOUNT_NAME == 'WMB 5K Simcards order')].head()

,MVNO_NAME,IMSI,MSISDN,S_IMSI,PROF_ID,SP_NAME,RESELLER_NAME,ACCOUNT_NAME,STATUS,START_DATE
1363304,NaN,425019613030000,NaN,260060149020000,NaN,NaN,WMB Limited RT,WMB 5K Simcards order,Active,2018-05-08 21:50:03.903
1363305,NaN,425019613030002,NaN,260060149020002,NaN,NaN,WMB Limited RT,WMB 5K Simcards order,Active,2018-05-08 21:50:04.153
1363306,NaN,425019613030008,NaN,260060149020008,NaN,NaN,WMB Limited RT,WMB 5K Simcards order,Active,2018-05-08 21:50:04.887
1363307,NaN,425019613030009,NaN,260060149020009,NaN,NaN,WMB Limited RT,WMB 5K Simcards order,Active,2018-05-08 21:50:04.997
1363308,NaN,425019613030010,NaN,260060149020010,NaN,NaN,WMB Limited RT,WMB 5K Simcards order,Active,2018-05-08 21:50:05.123


In [61]:
df_ocs_dmi.loc[df_ocs_dmi.IMSI.isin(df_dmi_n[(df_dmi_n.ACCOUNT_NAME == 'WMB 5K Simcards order')].IMSI),
              ['RESELLER_NAME', 'MVNO_NAME', 'IMSI', 'MSISDN', 'S_IMSI', 'SP_NAME', 'START_DATE']].\
sort_values(by=['IMSI', 'MSISDN'])\
.to_csv(join(downloads, 'WMB_5K_Simcards_order_Missed_DMI.csv'), index=False)

In [66]:
df_temp = df_ocs_dmi.loc[df_ocs_dmi.IMSI.isin(df_dmi_n[(df_dmi_n.ACCOUNT_NAME == 'WMB 5K Simcards order')].IMSI),
              ['RESELLER_NAME', 'MVNO_NAME', 'IMSI', 'MSISDN', 'S_IMSI', 'SP_NAME', 'START_DATE']].\
sort_values(by=['IMSI', 'MSISDN'])
s1 = df_temp.groupby('IMSI')['RESELLER_NAME'].count()
s1[s1.values > 2]

IMSI
425019613030302    3
425019613030522    3
425019613030524    3
425019613030525    3
425019613030533    3
425019613030534    3
425019613030591    3
425019613030592    3
Name: RESELLER_NAME, dtype: int64

In [69]:
imsi = '425019613030000'
print('Данные DMI:')
display(df_dmi[df_dmi.IMSI == imsi])
print('Данные OCS:')
display(df_ocs[df_ocs.IMSI == imsi])
print('Объединенные данные DMI и OCS:')
display(df_ocs_dmi[df_ocs_dmi.IMSI == imsi])

Данные DMI:


,MVNO_NAME,IMSI,MSISDN,S_IMSI,PROF_ID,SP_NAME
1376467,WMB_Combined,425019613030000,972540404500,425019613030000,1,Partner_WMB_Comb


Данные OCS:


,RESELLER_NAME,ACCOUNT_NAME,IMSI,S_IMSI,STATUS,START_DATE
672816,WMB Limited RT,WMB 5K Simcards order,425019613030000,425019613030000,Active,2018-05-08 21:50:03.903
672817,WMB Limited RT,WMB 5K Simcards order,425019613030000,260060149020000,Active,2018-05-08 21:50:03.903


Объединенные данные DMI и OCS:


,MVNO_NAME,IMSI,MSISDN,S_IMSI,PROF_ID,SP_NAME,RESELLER_NAME,ACCOUNT_NAME,STATUS,START_DATE
1360280,WMB_Combined,425019613030000,972540404500,425019613030000,1,Partner_WMB_Comb,WMB Limited RT,WMB 5K Simcards order,Active,2018-05-08 21:50:03.903
1363304,NaN,425019613030000,NaN,260060149020000,NaN,NaN,WMB Limited RT,WMB 5K Simcards order,Active,2018-05-08 21:50:03.903


In [32]:
df_dmi[(df_dmi.IMSI == imsi) & (df_dmi.S_IMSI == '260060149000082')].S_IMSI

126875    260060149000082
Name: S_IMSI, dtype: object

In [54]:
len(df_ocs[(df_ocs.IMSI == '425191200959682')].S_IMSI.values[1])

16

In [68]:
df_ocs[df_ocs.S_IMSI.apply(lambda x: len(str(x))) != 15]

,IMSI,S_IMSI,STATUS
459,89972011215043609895,89972011215043609895,Active
834,1234567890,1234567890,Active
1413,NaN,NaN,Active
2805,425191200959682,260060149000082,Active
13003,425019613005343,\r\r\n260060140009187,Active
334210,447781518786,447781518786,Active
336218,447781518787,447781518787,Active
336232,425000123456,425000123456,Active
451332,425191201134053,NaN,Active
650177,724540540268812,NaN,Active


1. В запрос DMI добавить статус из таблицы Sponsor, чтобы чтобы иметь возможность анализировать активных абонентов, а не Disconnected.
2. В запрос OCS добавить Suspended и Pending
3. Проверять данные DMI абонентов в состоянии Disconnected, но с не освобожденным MSISDN в таблице Subscriber
4. Добавить в запрос OCS время активации каждого IMSI, чтобы можно было добавлять отсутствующие IMSI по времени активации первого.
5. В запрос OCS добавить Reseller и Account, чтобы знать из каких реселлеров отсутствуют IMSI в DMI.